In [1]:
from data_utils import get_CIFAR10_data
import numpy as np

import keras
import tensorflow as tf
from keras.layers import Input, Conv2D, Dense, Flatten, Add, ZeroPadding2D, Merge, add, MaxPool2D
from keras.models import Model
from keras.utils.np_utils import to_categorical 
from tensorflow_weave import *

/Users/Tim/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Load the (preprocessed) CIFAR10 data.

data = get_CIFAR10_data()
for k, v in data.iteritems():
  print '%s: ' % k, v.shape

X_val:  (1000, 3, 32, 32)
X_train:  (49000, 3, 32, 32)
X_test:  (1000, 3, 32, 32)
y_val:  (1000,)
y_train:  (49000,)
y_test:  (1000,)


In [6]:
filter_size = 3
num_filters = 40
pad_size = filter_size//2


inputs = Input(shape=(3,32,32))
num_images = tf.shape(inputs)[0]

x_pad = ZeroPadding2D(padding=(pad_size,pad_size))(inputs)

x_per = Conv2D(num_filters,
               kernel_size = (3,3),
               strides=(1,1),
               padding='valid',
               activation='relu')(x_pad)
x_loc = Conv2D(num_filters,
               kernel_size= (3,3),
               strides=(1,1),
               padding='valid',
               activation = 'relu')(x_pad)


x_zero = ZeroWeave(num_img = num_images)(x_loc)
x_weave = ArrayWeave(num_img = num_images)(x_per)

x = Add()([x_weave, x_zero])

x = Conv2D(num_filters,
           kernel_size= (3,3),
           strides=(3,3),
           padding='valid',
           activation = 'relu')(x)

x = MaxPool2D()(x)
x = Flatten()(x)
x = Dense(100)(x)
predictions = Dense(10, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=[inputs], outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary

31


<bound method Model.summary of <keras.engine.training.Model object at 0x1816dd1dd0>>

In [7]:
num_train = 10*128
X_train = data['X_train'][:num_train]
Y_train = data['y_train'][:num_train]  

Y_train = to_categorical(Y_train, num_classes=10)

In [8]:
model.fit(X_train,
          Y_train,
          batch_size=16,
          epochs = 2,
          verbose=1)

Epoch 1/2
1280/1280 [==============================] - 96s 75ms/step - loss: 14.5464 - acc: 0.0961
Epoch 2/2
1280/1280 [==============================] - 66s 52ms/step - loss: 14.5441 - acc: 0.0977
